In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from dataflow import df_train, df_test, df_geography, df_product



In [2]:
import torch 

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.losses import MeanSquaredError

In [4]:
import sys
print("Python Interpreter Path:", sys.executable)


Python Interpreter Path: c:\Users\Hưng Tăng\AppData\Local\Programs\Python\Python312\python.exe


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Chuyển cột Date sang kiểu datetime
df_train['Date'] = pd.to_datetime(df_train['Date'], errors='coerce')
df_test['Date'] = pd.to_datetime(df_test['Date'], errors='coerce')

# Xử lý lỗi nếu có giá trị NaT
df_train.dropna(subset=['Date'], inplace=True)
df_test.dropna(subset=['Date'], inplace=True)

# Tính số ngày kể từ ngày đầu tiên trong tập train
min_date = df_train['Date'].min()
df_train['Date'] = (df_train['Date'] - min_date).dt.days
df_test['Date'] = (df_test['Date'] - min_date).dt.days

# Chia df_train thành train (80%) và val (20%)
df_train, df_val = train_test_split(df_train, test_size=0.2, shuffle=True, random_state=42)

# Chọn features và labels
features = ['ProductID', 'Date', 'Zip', 'COGS']
targets = ['Units', 'Revenue']

# Chuẩn hóa dữ liệu
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train = scaler_X.fit_transform(df_train[features])
X_val = scaler_X.transform(df_val[features])
X_test = scaler_X.transform(df_test[features])  # Dùng tập test để dự đoán

y_train = scaler_y.fit_transform(df_train[targets])
y_val = scaler_y.transform(df_val[targets])
y_test = scaler_y.transform(df_test[targets])  # Dùng để đánh giá mô hình


In [7]:
def create_sequences(X, y, time_steps=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i+time_steps])
        y_seq.append(y[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 20  # Dùng 20 ngày trước đó để dự đoán ngày tiếp theo

X_train_seq, y_train_seq = create_sequences(X_train, y_train, time_steps)
X_val_seq, y_val_seq = create_sequences(X_val, y_val, time_steps)
X_test_seq, y_test_seq = create_sequences(X_test, y_test, time_steps)  # Tập này sẽ dùng để dự đoán


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 🔹 Định nghĩa mô hình LSTM
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim1=64, hidden_dim2=32, output_dim=2):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim1, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim1, hidden_dim2, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_dim2, output_dim)
    
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout(x)
        x, _ = self.lstm2(x)
        x = self.dropout(x[:, -1, :])  # Lấy giá trị cuối cùng của chuỗi
        x = self.fc(x)
        return x

# 🔹 Hàm loss tách riêng Units và Revenue
def custom_loss(y_pred, y_true):
    mse = nn.MSELoss()
    loss_units = mse(y_pred[:, 0], y_true[:, 0])
    loss_revenue = mse(y_pred[:, 1], y_true[:, 1])
    return loss_units + loss_revenue

# 🔹 Chuyển dữ liệu sang tensor
X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val_seq, dtype=torch.float32)

# 🔹 Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 🔹 Khởi tạo mô hình
input_dim = X_train_seq.shape[2]
model = LSTMModel(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🔹 Callback in loss
for epoch in range(10):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = custom_loss(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # 🔹 Đánh giá trên tập validation
    model.eval()
    val_loss_units, val_loss_revenue = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            val_loss_units += nn.MSELoss()(y_pred[:, 0], y_batch[:, 0]).item()
            val_loss_revenue += nn.MSELoss()(y_pred[:, 1], y_batch[:, 1]).item()
    
    print(f"📊 Epoch {epoch+1} | Train Loss = {total_loss/len(train_loader):.4f} | "
          f"Val Loss Units = {val_loss_units/len(val_loader):.4f}, "
          f"Val Loss Revenue = {val_loss_revenue/len(val_loader):.4f}")


📊 Epoch 1 | Train Loss = 0.0003 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 2 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 3 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 4 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 5 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 6 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 7 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 8 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002
📊 Epoch 9 | Train Loss = 0.0002 | Val Loss Units = 0.0001, Val Loss Revenue = 0.0002


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

def evaluate_model(model, X_test_seq, y_test_seq, scaler_y, labels=["Units", "Revenue"]):
    """
    Dự đoán trên tập test, xử lý NaN, đánh giá bằng MAPE, RMSE, R^2 và vẽ biểu đồ đường.
    
    Args:
        model: Mô hình đã train.
        X_test_seq: Dữ liệu test đã được tạo sequence.
        y_test_seq: Ground truth của tập test.
        scaler_y: Bộ scaler dùng để đưa giá trị về dạng gốc.
        labels: Tên các chỉ số cần đánh giá (Units, Revenue).
    """
    # Dự đoán trên tập test
    y_pred_scaled = model.predict(X_test_seq)
    
    # Chuyển dữ liệu dự đoán và ground truth về dạng gốc
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_test_orig = scaler_y.inverse_transform(y_test_seq)

    # Kiểm tra NaN trong cả y_pred và y_test_orig
    if np.isnan(y_pred).any() or np.isnan(y_test_orig).any():
        print("⚠️ Cảnh báo: Có NaN trong dữ liệu! Sẽ thay thế bằng giá trị trung bình.")

        # Tính giá trị trung bình của từng cột, bỏ qua NaN
        col_means_pred = np.nanmean(y_pred, axis=0)
        col_means_test = np.nanmean(y_test_orig, axis=0)

        # Thay thế NaN trong y_pred
        for i in range(y_pred.shape[1]):
            y_pred[:, i] = np.where(np.isnan(y_pred[:, i]), col_means_pred[i], y_pred[:, i])

        # Thay thế NaN trong y_test_orig
        for i in range(y_test_orig.shape[1]):
            y_test_orig[:, i] = np.where(np.isnan(y_test_orig[:, i]), col_means_test[i], y_test_orig[:, i])

    # Đánh giá mô hình
    for i, label in enumerate(labels):
        # Tính toán các chỉ số đánh giá
        mape = mean_absolute_percentage_error(y_test_orig[:, i], y_pred[:, i])
        rmse = np.sqrt(mean_squared_error(y_test_orig[:, i], y_pred[:, i]))
        r2 = r2_score(y_test_orig[:, i], y_pred[:, i])

        # In kết quả đánh giá
        print(f"{label} - MAPE: {mape:.4f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")

        # Vẽ biểu đồ đường
        plt.figure(figsize=(10, 4))
        plt.plot(y_test_orig[:, i], label="Thực tế", linestyle='solid', color='blue')
        plt.plot(y_pred[:, i], label="Dự đoán", linestyle='dashed', color='red')
        plt.title(f"Biểu đồ dự đoán {label}")
        plt.xlabel("Thời gian")
        plt.ylabel(label)
        plt.legend()
        plt.grid(True)
        plt.show()


In [2]:
evaluate_model(model, X_test_seq, y_test_seq, scaler_y)


NameError: name 'model' is not defined

In [ ]:
# save model
model.save('lstm1_model.h5')

In [ ]:
import matplotlib.pyplot as plt

# Lấy dữ liệu loss từ quá trình huấn luyện
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Vẽ biểu đồ loss
plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Train Loss', marker='o')
plt.plot(val_loss, label='Validation Loss', marker='o')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss trong quá trình huấn luyện')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'history' is not defined